# Deploying pre-trained PyTorch vision models with Amazon SageMaker Neo On Inf1 Instance

Neo is a capability of Amazon SageMaker that enables machine learning models to train once and run anywhere in the cloud and at the edge. Inf1 instances are built from the ground up to support machine learning inference applications and feature up to 16 AWS Inferentia chips, high-performance machine learning inference chips designed and built by AWS. This notebook will show you how to deploy a pretrained PyTorch model to an Inf1 instance.

Please use sagemaker version at least 2.11.0 in order to support compile PyTorch model on Inf1 instances.

In [18]:
import sagemaker

print(sagemaker.__version__)

2.23.1


## Import ResNet18 from TorchVision

We'll import [ResNet18](https://arxiv.org/abs/1512.03385) model from TorchVision and create a model artifact `model.tar.gz`

In [4]:
import torch
import torchvision.models as models
import tarfile

resnet18 = models.resnet18(pretrained=True)
input_shape = [1, 3, 224, 224]
trace = torch.jit.trace(resnet18.float().eval(), torch.zeros(input_shape).float())
trace.save("model.pth")

with tarfile.open("model.tar.gz", "w:gz") as f:
    f.add("model.pth")
    f.add("resnet18.py")

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /home/ubuntu/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


--------

# Compile Model with Default Settings

We will forward the model artifact to Neo Compilation API. In this section, we will compile model with one core which is the default setting for compilation.

We will go through how to compile model for multiple cores using compiler options in the next section.

In [5]:
import boto3
import sagemaker
import time
from sagemaker.utils import name_from_base

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
region = sess.boto_region_name
bucket = sess.default_bucket()

compilation_job_name = name_from_base("TorchVision-ResNet18-Neo-Inf1")

model_key = "{}/model/model.tar.gz".format(compilation_job_name)
model_path = "s3://{}/{}".format(bucket, model_key)
boto3.resource("s3").Bucket(bucket).upload_file("model.tar.gz", model_key)
print("Uploaded model to s3:")
print(model_path)

sm_client = boto3.client("sagemaker")
compiled_model_path = "s3://{}/{}/output".format(bucket, compilation_job_name)
print("Output path for compiled model:")
print(compiled_model_path)

Uploaded model to s3:
s3://sagemaker-us-west-2-688520471316/TorchVision-ResNet18-Neo-Inf1-2021-05-28-21-03-56-772/model/model.tar.gz
Output path for compiled model:
s3://sagemaker-us-west-2-688520471316/TorchVision-ResNet18-Neo-Inf1-2021-05-28-21-03-56-772/output


We then create a PyTorchModel object, with default settings.

In [6]:
from sagemaker.pytorch.model import PyTorchModel

pytorch_model = PyTorchModel(
    model_data=model_path,
    role=role,
    entry_point="resnet18.py",
    framework_version="1.5.1",
    py_version="py3",
)

## Deploy model on Inf1 instance for real-time inferences

After creating the PyTorch model, we compile the model using Amazon SageMaker Neo to optize performance for our desired deployment target. To compile our model for deploying on Inf1 instances, we are using the  ``compile()`` method and select ``'ml_inf1'`` as our deployment target. The compiled model will then be deployed on an endpoint using Inf1 instances in Amazon SageMaker. 

## Compile the model 

The ``input_shape`` is the definition for the model's input tensor and ``output_path`` is where the compiled model will be stored in S3. **Important. If the following command result in a permission error, scroll up and locate the value of execution role returned by `get_execution_role()`. The role must have access to the S3 bucket specified in ``output_path``.**

In [7]:
neo_model = pytorch_model.compile(
    target_instance_family="ml_inf1",
    input_shape={"input0": [1, 3, 224, 224]},
    output_path=compiled_model_path,
    framework="pytorch",
    framework_version="1.5.1",
    role=role,
    job_name=compilation_job_name,
)

?...................................!

## Deploy the compiled model on a SageMaker endpoint

Now that we have the compiled model, we will deploy it on an Amazon SageMaker endpoint. Inf1 instances in Amazon SageMaker are available in four sizes: ml.inf1.xlarge, ml.inf1.2xlarge, ml.inf1.6xlarge, and ml.inf1.24xlarge. In this example, we are using ``'ml.inf1.xlarge'`` for deploying our model.

In [8]:
predictor = neo_model.deploy(instance_type="ml.inf1.xlarge", initial_instance_count=1)

-------------!

In [9]:
predictor.endpoint_name

'sagemaker-neo-pytorch-ml-inf1-2021-05-28-21-07-15-912'

## Invoking the endpoint

Once the endpoint is ready, you can send requests to it and receive inference results in real-time with low latency. 

Let's try to send a cat picture.

![title](cat.jpg)

In [10]:
import json
import numpy as np

sm_runtime = boto3.Session().client("sagemaker-runtime")

with open("cat.jpg", "rb") as f:
    payload = f.read()

response = sm_runtime.invoke_endpoint(
    EndpointName=predictor.endpoint_name, ContentType="application/x-image", Body=payload
)
print(response)
result = json.loads(response["Body"].read().decode())
print("Most likely class: {}".format(np.argmax(result)))

{'ResponseMetadata': {'RequestId': 'f2a70e61-dfd8-4826-9ed8-47addbf90d57', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'f2a70e61-dfd8-4826-9ed8-47addbf90d57', 'x-amzn-invoked-production-variant': 'AllTraffic', 'date': 'Fri, 28 May 2021 21:13:53 GMT', 'content-type': 'application/json', 'content-length': '10863'}, 'RetryAttempts': 0}, 'ContentType': 'application/json', 'InvokedProductionVariant': 'AllTraffic', 'Body': <botocore.response.StreamingBody object at 0x7f0559ba57f0>}
Most likely class: 282


In [11]:
# Load names for ImageNet classes
object_categories = {}
with open("imagenet1000_clsidx_to_labels.txt", "r") as f:
    for line in f:
        key, val = line.strip().split(":")
        object_categories[key] = val
print(
    "Result: label - "
    + object_categories[str(np.argmax(result))]
    + " probability - "
    + str(np.amax(result))
)

Result: label -  'tiger cat', probability - 15.125


## Delete the Endpoint
Having an endpoint running will incur some costs. Therefore as a clean-up job, we should delete the endpoint.

In [12]:
sess.delete_endpoint(predictor.endpoint_name)

------------------------